In [20]:
import pandas as pd
import numpy as np
from docx import Document
from pathlib import Path
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

from typing import List, Dict
from datetime import datetime
import sys
import os
import re

In [21]:

def get_dataframe_summary(df):
    """
    Returns a summary DataFrame for the given DataFrame.
    
    The summary includes:
      - Data Type
      - Non Null Count
      - Null Count
      - Null Percentage
      - Unique Values count
    """
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    summary_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non Null Count': df.count(),
        'Null Count': df.isna().sum(),
        'Null Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Unique Values': [df[col].nunique() for col in df.columns],
    })
    
    return summary_df

In [22]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[8]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed

from app.modules.sga.minpub.report_validator.service.objetivos.calculations import extract_indisponibilidad_anexos

from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_corte_excel
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_335
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_380
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_df_cid_cuismp_sharepoint
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_df_word_datos_anexos_indis
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_sga_335_corte_excel_sharepoint_cuismp_sga380 import merge_sga_335_corte_excel_sharepoint_cuismp_sga380
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_datos_anexos_disponibilidad_dfs_merged_sga import merge_word_datos_anexos_disponibilidad_df_merged_sga
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_3.o1_anexos_sustentos_paradas_reloj_validator import validate_anexos_indisponibilidad_word
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_3.o1_anexos_sustentos_paradas_reloj_validator import build_failure_messages_validate_anexos_indisponibilidad_word


In [23]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 2.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_30-03-2025_06-04-2025_20250410_173936.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_30-03-2025_04-04-2025_20250410_195338.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS - CORTE 2.docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS - CORTE 2.docx"



In [24]:
df_corte_excel = pd.read_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=2, engine="openpyxl")
df_sga_dinamico_335 = pd.read_excel(SAVE_DIR_EXTRACT_SGA_335) 
df_sga_dinamico_380 = pd.read_excel(DIR_PARADAS_RELOJ)
df_cid_cuismp_sharepoint = pd.read_excel(CID_CUISMP_PATH)


df_word_datos_anexos_indis =  extract_indisponibilidad_anexos(DIR_WORD_DATOS)
df_word_telefonia_anexos_indis = extract_indisponibilidad_anexos(DIR_WORD_TELEFONIA)



In [25]:
df_word_datos_anexos_indis

,ticket,indisponibilidad_header,indisponibilidad_periodos,indisponibilidad_total
0,21798497,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),3/04/2025 13:09:00 hasta el día 3/04/2025 20:00:00,(Total de horas sin acceso a la sede: 6:51 horas)
1,21791018,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),20/03/2025 19:00:00 hasta el día 24/03/2025 13:00:00,(Total de horas sin acceso a la sede: 90:00 horas)
2,21795319,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),28/03/2025 15:17:00 hasta el día 31/03/2025 13:00:00,(Total de horas sin acceso a la sede: 69:43 horas)
3,21796264,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),31/03/2025 08:55:00 hasta el día 2/04/2025 11:54:00,(Total de horas sin acceso a la sede: 50:59 horas)
4,21796323,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),31/03/2025 10:40:00 hasta el día 31/03/2025 16:00:00,(Total de horas sin acceso a la sede: 05:20 horas)
5,21796951,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),1/04/2025 11:41:00 hasta el día 2/04/2025 13:00:00,(Total de horas sin acceso a la sede: 25:19 horas)
6,21789943,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),18/03/2025 17:24:00 hasta el día 19/03/2025 07:00:00,(Total de horas sin acceso a la sede: 13:36 horas)
7,21796956,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),01/04/2025 12:57:00 hasta el día 01/04/2025 14:46:00,(Total de horas sin acceso a la sede: 1:49 horas)
8,21796590,,,
9,21796938,,,


In [26]:

df_process_word = preprocess_df_word_datos_anexos_indis(df_word_datos_anexos_indis)
summary = get_dataframe_summary(df_process_word)
summary

,Data Type,Non Null Count,Null Count,Null Percentage,Unique Values
nro_incidencia,object,27,0,0.00,27
indisponibilidad_header,object,27,0,0.00,2
indisponibilidad_periodos,object,27,0,0.00,9
indisponibilidad_total,object,27,0,0.00,9


In [27]:

df_preprocss_excel = preprocess_corte_excel(df_corte_excel)
df_preprocss_sharepoint_cuismp = preprocess_df_cid_cuismp_sharepoint(df_cid_cuismp_sharepoint)
df_preprocss_sga335 = preprocess_335(df_sga_dinamico_335)
df_preprocss_sga380 = preprocess_380(df_sga_dinamico_380)



In [28]:
df_process_word.head(1)

,nro_incidencia,indisponibilidad_header,indisponibilidad_periodos,indisponibilidad_total
0,21798497,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),3/04/2025 13:09:00 hasta el día 3/04/2025 20:00:00,(Total de horas sin acceso a la sede: 6:51 horas)


In [29]:
df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_preprocss_excel, df_preprocss_sga335,
        df_preprocss_sharepoint_cuismp, df_preprocss_sga380,
        'both'
        )
df_matched_corte_sga335_Sharepoint_cuismp_sga380.head(1)
df_matched_corte_sga335_Sharepoint_cuismp_sga380[df_matched_corte_sga335_Sharepoint_cuismp_sga380['nro_incidencia']=='21798497']

,nro_incidencia,FECHA Y HORA INICIO,FECHA Y HORA FIN,ESTATUS,SERVICIO,CUISMP_sga_dinamico_335_excel_matched,TIPO CASO,AVERÍA,TIEMPO (HH:MM),COMPONENTE,DF,DIRECCION,OBSERVACIÓN,CID,FIN-INICIO (HH:MM),DETERMINACIÓN DE LA CAUSA,RESPONSABILIDAD,TIPO REPORTE,Duracion entero,Agrupación entero,USUARIO,ESPECIALISTA,CODINCIDENCEPADRE,MASIVO,MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS,TIPO DE INCIDENCIA,TIEMPO INTERRUPCION,INDISPONIBILIDAD,DC + INDISPONIBILIDAD,TIEMPO (HH:MM)_trimed,FIN-INICIO (HH:MM)_trimed,FECHA_Y_HORA_INICIO_fmt,FECHA_Y_HORA_FIN_fmt,canal_ingreso,filter,codincidencepadre,masivo,hijos,pxav,estado,mesa_atencion,catg,ubigeo,cnoci,departamento,distrito,caso,tipo_caso,nombre_cliente,codigo_cliente,sector,segm,cid,mediotxcid,mediotxide,mediotxsot,mediotx,direccion,tipopy,srv,um,dias_transcurridos,considerar,comentario,otpint,sotpint,fecgensotpint,pausacliente2pint,otpext,sotpext,fecgensotpext,sotwimax,sotlte,crq,cantidadcrq,seqcrq,crqobs,remedy,site,cantidadremedy,seqremedy,remedyobs,sot,usot,feccomsot,sot_comentario,compute_0055,fo,mo,slacomp,desp,desplazamiento,horas_pendiente,horas_sin_update,tipo_incidencia,tipo_servicio,producto,bandeja_cnoc,dpto_actual,fecha_comunicacion_cliente,fecha_generacion,fg_padre,fecha_instalacion,fecha_apertura,mes_apertura,anio_apertura,dia_apertura,semana_del_mes,hora,hl,dpto_origen,dpto_origen_detalle,area_1ra_anotacion,area_que_derivo_al_cnoc,derivadoalcnoc,demora_derivacion,demora_tomar_ticket,usuariotomaticket,proactivo,usuario_actual,usuario_ultima_anotacion,compute_0090,compute_0091,derivacion_del_operador,area_ultima_anotacion,fecha_derivacion_cierre,mes_derivacion_cierre,dia_derivacion_cierre,usuario_cierra,tipo_usuario_cierra,tipificacion_interrupcion,tip_interr_filtrado,interrupcion_inicio,interrupcion_fin,tiempocliente,tiempo_interrupcion,compute_0105,compute_0106,compute_0107,compute_0108,compute_0109,compute_0110,compute_0111,compute_0112,tiempopint,tiempored,tiempoproveedor,tiempopext,cruce,tipificacion_problema,tipificacion_observacion,tipificacion_tipo,tipificacion_responsable,tiempotransitocnoc,it_userid,it_responsable,it_determinacion_de_la_causa,it_medidas_tomadas,it_conclusiones,reinc,hora_sistema,_merge,CUISMP_sharepoint_cid_cuismp,Distrito Fiscal,%Disponibilidad,BW contratado,SEDE,CID NUEVO,Unnamed: 7,Unnamed: 8,Unnamed: 9,sum_paradas,clock_stops,clock_stops_paragraph,clock_stops_paragraph_header,clock_stops_paragraph_periodos,clock_stops_paragraph_footer
114,21798497,2025-04-03 12:55:00,2025-04-04 05:00:00,Caso Concluído,Red Privada Virtual Local,30008,SIN SERVICIO,PUERTO BLOQUEADO,03:14,COMPONENTE II,Selva Central,"SN . . SIN NOMBRE Jr. Francisco Irazola Nº 618 - 2do Piso / DML / -11.257260,-74.640123",Se generó ticket para la revisión del servicio de datos de la sede Satipo2,21097227,16:05,El inconveniente se originó por un problema de bloqueo logico del puerto WAN del IDU CLIENTE,CLARO,RECLAMO,3,Entre 1h a 4h,E755076,CLARO - CORTE,21798497,No,"El cliente, la Srta. Maria Onofre, reportó la interrupción total de su servicio de datos identificado con el CUISMP 30008 se genero ticket 21798497 el día 03/04/2025 a las 12:55 horas, . Inmediatamente, Claro revisó el enlace encontrando un inconveniente con uno de los equipos de red ubicado en la sede del cliente. Posteriormente, se determino que la afectación se originó debido a un bloqueo lógico en el equipo de red (IDU CLIENTE) ubicado en la sede del cliente, por lo que se realizó un desplazamiento de personal tecnico especializado a la sede del cliente el día 04/04/2025 02:30 hrs, de acuerdo con la disponibilidad de tiempo del cliente, para proceder con los correctivos necesarios realizando el reinicio de dicho equipo de red y revisando conexiones físicas en sede. Finalmente, luego de los correctivos, se verificó el restablecimiento del servicio el 04/04/2025 a las 05:00 horas, comprobándose el correcto funcionamiento y estabilidad del mismo.",REPORTE PREVIO - Calidad,"3,22",,El inconveniente se originó por un problema 

In [30]:
merge_word_dfs_merged_anexos_dato = merge_word_datos_anexos_disponibilidad_df_merged_sga(
    df_process_word,
    df_matched_corte_sga335_Sharepoint_cuismp_sga380,
      'both' )
merge_word_dfs_merged_anexos_dato.head(1)

,nro_incidencia,indisponibilidad_header,indisponibilidad_periodos,indisponibilidad_total,FECHA Y HORA INICIO,FECHA Y HORA FIN,ESTATUS,SERVICIO,CUISMP_sga_dinamico_335_excel_matched,TIPO CASO,AVERÍA,TIEMPO (HH:MM),COMPONENTE,DF,DIRECCION,OBSERVACIÓN,CID,FIN-INICIO (HH:MM),DETERMINACIÓN DE LA CAUSA,RESPONSABILIDAD,TIPO REPORTE,Duracion entero,Agrupación entero,USUARIO,ESPECIALISTA,CODINCIDENCEPADRE,MASIVO,MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS,TIPO DE INCIDENCIA,TIEMPO INTERRUPCION,INDISPONIBILIDAD,DC + INDISPONIBILIDAD,TIEMPO (HH:MM)_trimed,FIN-INICIO (HH:MM)_trimed,FECHA_Y_HORA_INICIO_fmt,FECHA_Y_HORA_FIN_fmt,canal_ingreso,filter,codincidencepadre,masivo,hijos,pxav,estado,mesa_atencion,catg,ubigeo,cnoci,departamento,distrito,caso,tipo_caso,nombre_cliente,codigo_cliente,sector,segm,cid,mediotxcid,mediotxide,mediotxsot,mediotx,direccion,tipopy,srv,um,dias_transcurridos,considerar,comentario,otpint,sotpint,fecgensotpint,pausacliente2pint,otpext,sotpext,fecgensotpext,sotwimax,sotlte,crq,cantidadcrq,seqcrq,crqobs,remedy,site,cantidadremedy,seqremedy,remedyobs,sot,usot,feccomsot,sot_comentario,compute_0055,fo,mo,slacomp,desp,desplazamiento,horas_pendiente,horas_sin_update,tipo_incidencia,tipo_servicio,producto,bandeja_cnoc,dpto_actual,fecha_comunicacion_cliente,fecha_generacion,fg_padre,fecha_instalacion,fecha_apertura,mes_apertura,anio_apertura,dia_apertura,semana_del_mes,hora,hl,dpto_origen,dpto_origen_detalle,area_1ra_anotacion,area_que_derivo_al_cnoc,derivadoalcnoc,demora_derivacion,demora_tomar_ticket,usuariotomaticket,proactivo,usuario_actual,usuario_ultima_anotacion,compute_0090,compute_0091,derivacion_del_operador,area_ultima_anotacion,fecha_derivacion_cierre,mes_derivacion_cierre,dia_derivacion_cierre,usuario_cierra,tipo_usuario_cierra,tipificacion_interrupcion,tip_interr_filtrado,interrupcion_inicio,interrupcion_fin,tiempocliente,tiempo_interrupcion,compute_0105,compute_0106,compute_0107,compute_0108,compute_0109,compute_0110,compute_0111,compute_0112,tiempopint,tiempored,tiempoproveedor,tiempopext,cruce,tipificacion_problema,tipificacion_observacion,tipificacion_tipo,tipificacion_responsable,tiempotransitocnoc,it_userid,it_responsable,it_determinacion_de_la_causa,it_medidas_tomadas,it_conclusiones,reinc,hora_sistema,_merge,CUISMP_sharepoint_cid_cuismp,Distrito Fiscal,%Disponibilidad,BW contratado,SEDE,CID NUEVO,Unnamed: 7,Unnamed: 8,Unnamed: 9,sum_paradas,clock_stops,clock_stops_paragraph,clock_stops_paragraph_header,clock_stops_paragraph_periodos,clock_stops_paragraph_footer,merge_flag_datos
0,21798497,Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s),3/04/2025 13:09:00 hasta el día 3/04/2025 20:00:00,(Total de horas sin acceso a la sede: 6:51 horas),2025-04-03 12:55:00,2025-04-04 05:00:00,Caso Concluído,Red Privada Virtual Local,30008,SIN SERVICIO,PUERTO BLOQUEADO,03:14,COMPONENTE II,Selva Central,"SN . . SIN NOMBRE Jr. Francisco Irazola Nº 618 - 2do Piso / DML / -11.257260,-74.640123",Se generó ticket para la revisión del servicio de datos de la sede Satipo2,21097227,16:05,El inconveniente se originó por un problema de bloqueo logico del puerto WAN del IDU CLIENTE,CLARO,RECLAMO,3,Entre 1h a 4h,E755076,CLARO - CORTE,21798497,No,"El cliente, la Srta. Maria Onofre, reportó la interrupción total de su servicio de datos identificado con el CUISMP 30008 se genero ticket 21798497 el día 03/04/2025 a las 12:55 horas, . Inmediatamente, Claro revisó el enlace encontrando un inconveniente con uno de los equipos de red ubicado en la sede del cliente. Posteriormente, se determino que la afectación se originó debido a un bloqueo lógico en el equipo de red (IDU CLIENTE) ubicado en la sede del cliente, por lo que se realizó un desplazamiento de personal tecnico especializado a la sede del cliente el día 04/04/2025 02:30 hrs, de acuerdo con la disponibilidad de tiempo del cliente, para proceder con los correctivos necesarios realizando el reinicio de dicho equipo de red y revisando conexiones

In [31]:
#df_process_word

In [32]:
summa_word = get_dataframe_summary(df_process_word)
summa_word

,Data Type,Non Null Count,Null Count,Null Percentage,Unique Values
nro_incidencia,object,27,0,0.00,27
indisponibilidad_header,object,27,0,0.00,2
indisponibilidad_periodos,object,27,0,0.00,9
indisponibilidad_total,object,27,0,0.00,9


In [33]:
sum_excel = get_dataframe_summary(df_preprocss_excel)
#sum_excel

In [34]:
row = df_preprocss_excel[df_preprocss_excel['nro_incidencia'] == '21798497']
#row

In [35]:
df_vali = validate_anexos_indisponibilidad_word(merge_word_dfs_merged_anexos_dato, 'COMPONENTE II')
#df_vali.head(5)
#df_vali[['nro_incidencia','indisponibilidad_extract','expected_indisponibilidad']]

In [36]:
df_mess = build_failure_messages_validate_anexos_indisponibilidad_word(df_vali)
df_mess

,nro_incidencia,mensaje,TIPO REPORTE,objetivo
0,21798497,No coincide paradas de reloj de word indisponibilidad en anexos : 3/04/2025 13:09:00 hasta el día 3/04/2025 20:00:00 es diferente a sga : 03/04/2025 13:09 hasta el día 04/04/2025 02:00. No coincide total horas sin acceso a la sede de word indisponibilidad en anexos : (Total de horas sin acceso a la sede: 6:51 horas) es diferente a sga : (Total de horas sin acceso a la sede: 12:51 horas).,RECLAMO,3.10
1,21791018,No coincide texto inicio de word indisponibilidad en anexos : Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s) es diferente a sga : . No coincide paradas de reloj de word indisponibilidad en anexos : 20/03/2025 19:00:00 hasta el día 24/03/2025 13:00:00 es diferente a sga : . No coincide total horas sin acceso a la sede de word indisponibilidad en anexos : (Total de horas sin acceso a la sede: 90:00 horas) es diferente a sga : .,RECLAMO,3.10
2,21795319,No coincide paradas de reloj de word indisponibilidad en anexos : 28/03/2025 15:17:00 hasta el día 31/03/2025 13:00:00 es diferente a sga : 28/03/2025 15:17 hasta el día 31/03/2025 13:00.,RECLAMO,3.10
3,21796264,No coincide paradas de reloj de word indisponibilidad en anexos : 31/03/2025 08:55:00 hasta el día 2/04/2025 11:54:00 es diferente a sga : 31/03/2025 08:55 hasta el día 02/04/2025 11:54.,RECLAMO,3.10
4,21796323,No coincide paradas de reloj de word indisponibilidad en anexos : 31/03/2025 10:40:00 hasta el día 31/03/2025 16:00:00 es diferente a sga : 31/03/2025 10:40 hasta el día 31/03/2025 16:00.,RECLAMO,3.10
5,21796951,No coincide paradas de reloj de word indisponibilidad en anexos : 1/04/2025 11:41:00 hasta el día 2/04/2025 13:00:00 es diferente a sga : 01/04/2025 11:41 hasta el día 02/04/2025 13:00.,RECLAMO,3.10
6,21789943,No coincide texto inicio de word indisponibilidad en anexos : Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s) es diferente a sga : . No coincide paradas de reloj de word indisponibilidad en anexos : 18/03/2025 17:24:00 hasta el día 19/03/2025 07:00:00 es diferente a sga : . No coincide total horas sin acceso a la sede de word indisponibilidad en anexos : (Total de horas sin acceso a la sede: 13:36 horas) es diferente a sga : .,RECLAMO,3.10
7,21796956,No coincide paradas de reloj de word indisponibilidad en anexos : 01/04/2025 12:57:00 hasta el día 01/04/2025 14:46:00 es diferente a sga : 01/04/2025 09:10 hasta el día 01/04/2025 14:46. No coincide total horas sin acceso a la sede de word indisponibilidad en anexos : (Total de horas sin acceso a la sede: 1:49 horas) es diferente a sga : (Total de horas sin acceso a la sede: 05:36 horas).,RECLAMO,3.10
